In [25]:
import pandas as pd
import numpy as np

In [26]:
df = pd.read_csv("spam.csv")
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [27]:
df.Category.value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [28]:
df['spam'] = df['Category'].apply(lambda x: 1 if x =='spam' else 0)
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.Message, df.spam, test_size=0.2)

### CountVectorizer to create BOW

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

v = CountVectorizer()

X_train_cv = v.fit_transform(X_train.values)
X_train_cv

<4457x7805 sparse matrix of type '<class 'numpy.int64'>'
	with 59270 stored elements in Compressed Sparse Row format>

In [31]:
v.get_feature_names_out()[1771]

'cheaper'

In [32]:
v.vocabulary_

{'same': 5947,
 'to': 6996,
 'dunno': 2512,
 'leh': 4096,
 'cant': 1664,
 'remember': 5734,
 'mayb': 4426,
 'lor': 4233,
 'so': 6350,
 'wat': 7454,
 'time': 6964,
 'we': 7472,
 'meeting': 4451,
 'tmr': 6991,
 'mm': 4554,
 'feeling': 2831,
 'sleepy': 6288,
 'today': 7001,
 'itself': 3793,
 'shall': 6122,
 'get': 3138,
 'that': 6870,
 'dear': 2214,
 'know': 3987,
 'grumpy': 3276,
 'old': 4954,
 'people': 5178,
 'my': 4690,
 'mom': 4587,
 'was': 7448,
 'like': 4139,
 'you': 7764,
 'better': 1349,
 'not': 4854,
 'be': 1290,
 'lying': 4301,
 'then': 6888,
 'again': 870,
 'am': 945,
 'always': 943,
 'the': 6874,
 'one': 4966,
 'play': 5275,
 'jokes': 3872,
 'now': 4869,
 'have': 3382,
 'started': 6521,
 'dont': 2430,
 'stop': 6562,
 'just': 3903,
 'pray': 5404,
 'for': 2966,
 'more': 4608,
 'good': 3204,
 'ideas': 3615,
 'and': 975,
 'anything': 1016,
 'see': 6043,
 'can': 1652,
 'help': 3426,
 'guys': 3305,
 'll': 4185,
 'forward': 2989,
 'link': 4157,
 'in': 3666,
 'xam': 7703,
 'hall': 33

In [33]:
X_train_np = X_train_cv.toarray()
X_train_np[0]

array([0, 0, 0, ..., 0, 0, 0])

In [34]:
np.where(X_train_np[0]!=0)

(array([5947, 6996]),)

In [37]:
X_train[:4][4856]

/tmp/ipykernel_17247/90030403.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  X_train[:4][4856]


'Same to u...'

In [38]:
X_train_np[0][4856]

0

### Train Naive Bayes Model

In [39]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train_cv, y_train)

MultinomialNB()

In [40]:
X_test_cv = v.transform(X_test)

In [41]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test_cv)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       977
           1       0.98      0.93      0.95       138

    accuracy                           0.99      1115
   macro avg       0.98      0.96      0.97      1115
weighted avg       0.99      0.99      0.99      1115



In [42]:
emails = [
    'Hey mohan, can we get together to watch footbal game tomorrow?',
    'Upto 20% discount on parking, exclusive offer just for you. Dont miss this reward!'
]

emails_count = v.transform(emails)
model.predict(emails_count)

array([0, 1])

### Train the model using sklearn pipeline and reduce number of lines of code

In [43]:
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])
clf.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [44]:
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       977
           1       0.98      0.93      0.95       138

    accuracy                           0.99      1115
   macro avg       0.98      0.96      0.97      1115
weighted avg       0.99      0.99      0.99      1115

